In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None #huh?


In [18]:
df=pd.read_csv("week_12_match_upload.csv")
df['date']=pd.to_datetime(df['date'])

In [19]:
df.dtypes

Unnamed: 0             int64
id                     int64
team_one              object
team_two              object
stage                 object
winner                object
date          datetime64[ns]
dtype: object

In [20]:
#Set the corona_virus_isolation feature.  This will be True if games are being played while in isolation because of the pandemic
df['corona_virus_isolation'] = False

#We will use a mask to properly set the feature:
mask = (df['date'] > '03/20/2020')
df['corona_virus_isolation'] = mask

In [21]:
#Initiate some features:
df['t1_wins_season'] = ""
df['t1_losses_season'] = ""
df['t2_wins_season'] = ""
df['t2_losses_season'] = ""
df['t1_matches_season'] = ""
df['t2_matches_season'] = ""
df['t1_win_percent_season'] = ""
df['t2_win_percent_season'] = ""
df['t1_wins_alltime'] = ""
df['t1_losses_alltime'] = ""
df['t2_wins_alltime'] = ""
df['t2_losses_alltime'] = ""
df['t1_matches_alltime'] = ""
df['t2_matches_alltime'] = ""
df['t1_win_percent_alltime'] = ""
df['t2_win_percent_alltime'] = ""
df['t1_wins_last_3'] = ""
df['t1_losses_last_3'] = ""
df['t2_wins_last_3'] = ""
df['t2_losses_last_3'] = ""
df['t1_win_percent_last_3'] = ""
df['t2_win_percent_last_3'] = ""
df['t1_wins_last_5'] = ""
df['t1_losses_last_5'] = ""
df['t2_wins_last_5'] = ""
df['t2_losses_last_5'] = ""
df['t1_win_percent_last_5'] = ""
df['t2_win_percent_last_5'] = ""
df['t1_wins_last_10'] = ""
df['t1_losses_last_10'] = ""
df['t2_wins_last_10'] = ""
df['t2_losses_last_10'] = ""
df['t1_win_percent_last_10'] = ""
df['t2_win_percent_last_10'] = ""
df['t1_wins_last_20'] = ""
df['t1_losses_last_20'] = ""
df['t2_wins_last_20'] = ""
df['t2_losses_last_20'] = ""
df['t1_win_percent_last_20'] = ""
df['t2_win_percent_last_20'] = ""
df['t1_place_last_season'] = ""
df['t2_place_last_season'] = ""
df['t1_wins_vs_t2'] = ""
df['t1_losses_vs_t2'] = ""
df['t1_matches_vs_t2'] = ""
df['t1_win_percent_vs_t2'] = ""
df['winner_label'] = ""

In [22]:
def set_winners(master_df):
    id_list = master_df.id.unique()

    for i in id_list:
        t1 = ((master_df[master_df['id'] == i]).team_one).iloc[0]
        t2 = ((master_df[master_df['id'] == i]).team_two).iloc[0]
        winner = ((master_df[master_df['id'] == i]).winner).iloc[0]
        mask = master_df['id'] == i
        if t1 == winner:
            master_df['winner_label'][mask] = 0
        elif t2 == winner:
            master_df['winner_label'][mask] = 1
    
    return master_df

In [23]:
def get_win_loss_total(wl_df, wl_id):
    t1 = ((wl_df[wl_df['id'] == wl_id]).team_one).iloc[0]
    t2 = ((wl_df[wl_df['id'] == wl_id]).team_two).iloc[0]
    date = ((wl_df[wl_df['id'] == wl_id]).date).iloc[0]
    #print(f"Team 1: {t1} Team 2: {t2} Date: {date}")
    teams=[t1, t2]
    
    return_info=[]
    
    for team in teams:
        df_partial_season=wl_df[wl_df['date']<date]
        season_1 = df_partial_season[df_partial_season['team_one'] == team]
        season_2 = df_partial_season[df_partial_season['team_two'] == team]
        team_season = season_1.append(season_2)
        wins = len(team_season[team_season['winner'] == team])
        losses = len(team_season[team_season['winner'] != team])
        matches = wins+losses
        #print(f"{team} {len(season_1)} {len(season_2)} {len(team_season)} {wins} ")
        return_info.append(wins)
        return_info.append(losses)
        return_info.append(matches)
        
    return(return_info)

In [24]:
def find_last_season_results(master_df):
    results_2018 = {'Philadelphia Fusion':6, 
                   'New York Excelsior':1, 
                   'Seoul Dynasty':8,
                   'Shanghai Dragons':12,
                   'Toronto Defiant': '', 
                   'Atlanta Reign': '', 
                   'Dallas Fuel': 10,
                   'Chengdu Hunters': '', 
                   'London Spitfire': 5, 
                   'Washington Justice': '',
                   'Los Angeles Valiant': 2, 
                   'Vancouver Titans': '', 
                   'Houston Outlaws': 7,
                   'San Francisco Shock': 9, 
                   'Guangzhou Charge': '', 
                   'Los Angeles Gladiators': 4,
                   'Hangzhou Spark': '', 
                   'Florida Mayhem': 11, 
                   'Paris Eternal': '', 
                   'Boston Uprising': 3     
    }
    results_2019 = {
        'Toronto Defiant':17, 'London Spitfire':7, 'Los Angeles Gladiators':5,
 'Los Angeles Valiant':13, 'Boston Uprising':19, 'San Francisco Shock':3,
 'Florida Mayhem':20, 'Washington Justice':17, 'New York Excelsior':2,
 'Paris Eternal':14, 'Guangzhou Charge':9, 'Chengdu Hunters':12, 'Hangzhou Spark':4,
 'Seoul Dynasty':8, 'Shanghai Dragons':11, 'Houston Outlaws':16,
 'Philadelphia Fusion':10, 'Dallas Fuel':15, 'Vancouver Titans':1, 'Atlanta Reign':6
        
    }
    
    
    years = ['2019', '2020']
    for year in years:
        if year == '2019':
            min_date = '01/01/2019'
            max_date = '12/31/2019'
            
        elif year == '2020':
            min_date = '01/01/2020'
            max_date = '12/31/2020'
            
        df_reduced = master_df[master_df['date'] > min_date]
        df_reduced = df_reduced[df_reduced['date'] < max_date]        
        #print(df_reduced.team_one.unique())
        id_list=df_reduced.id.unique()
        for i in id_list:
            t1 = ((df_reduced[df_reduced['id'] == i]).team_one).iloc[0]
            t2 = ((df_reduced[df_reduced['id'] == i]).team_two).iloc[0]
            mask = master_df['id'] == i
            if year=='2019':
                master_df['t1_place_last_season'][mask] = results_2018.get(t1)
                master_df['t2_place_last_season'][mask] = results_2018.get(t2)
            if year=='2020':
                master_df['t1_place_last_season'][mask] = results_2019.get(t1)
                master_df['t2_place_last_season'][mask] = results_2019.get(t2)
                
    return master_df

In [25]:
def find_head_to_head_results(master_df):
    id_list = master_df.id.unique()

    for i in id_list:
        t1 = ((master_df[master_df['id'] == i]).team_one).iloc[0]
        t2 = ((master_df[master_df['id'] == i]).team_two).iloc[0]
        date = ((master_df[master_df['id'] == i]).date).iloc[0]
        
        
        print(f"Team 1: {t1} Team 2: {t2} Date: {date}")

        df_partial_season=master_df[master_df['date']<date]
        season_1 = df_partial_season[df_partial_season['team_one'] == t1]
        season_1 = season_1[season_1['team_two'] == t2]        
        season_2 = df_partial_season[df_partial_season['team_two'] == t1]
        season_2 = season_2[season_2['team_one'] == t2]
        team_season = season_1.append(season_2)
        wins = len(team_season[team_season['winner'] == t1])
        losses = len(team_season[team_season['winner'] != t1])
        matches = wins+losses

        mask = master_df['id'] == i
        master_df['t1_wins_vs_t2'][mask] = wins
        master_df['t1_losses_vs_t2'][mask] = losses
        master_df['t1_matches_vs_t2'][mask] = matches   
        print(f"{t1} vs {t2} on {date} record: {wins} - {losses} ")
        if matches > 0:
            master_df['t1_win_percent_vs_t2'][mask] = wins/matches

        
    return master_df

In [26]:
def find_team_match_results(master_df, min_date, max_date, type):
    df_reduced = master_df[master_df['date'] > min_date]
    df_reduced = df_reduced[df_reduced['date'] < max_date]
    
    #We can use this to make an ID list...
    id_list = df_reduced.id.unique()
    
    for i in id_list:
        win_loss_list = (get_win_loss_total(df_reduced, i))
        mask = master_df['id'] == i

        #calculate the win percentages for each match
        if win_loss_list[2] == 0:
            t1_win_percent = 0
        else:
            t1_win_percent = win_loss_list[0] / win_loss_list[2]
        if win_loss_list[5] == 0:
            t2_win_percent = 0
        else:
            t2_win_percent = win_loss_list[3] / win_loss_list[5]
        
        
        if type=='current_season':
            master_df['t1_wins_season'][mask] = win_loss_list[0]
            master_df['t1_losses_season'][mask] = win_loss_list[1]
            master_df['t1_matches_season'][mask] = win_loss_list[2]
            master_df['t1_win_percent_season'][mask] = t1_win_percent
            master_df['t2_wins_season'][mask] = win_loss_list[3]
            master_df['t2_losses_season'][mask] = win_loss_list[4]
            master_df['t2_matches_season'][mask] = win_loss_list[5]  
            master_df['t2_win_percent_season'][mask] = t2_win_percent            
        if type=='all_time':
            master_df['t1_wins_alltime'][mask] = win_loss_list[0]
            master_df['t1_losses_alltime'][mask] = win_loss_list[1]
            master_df['t1_matches_alltime'][mask] = win_loss_list[2]
            master_df['t1_win_percent_alltime'][mask] = t1_win_percent
            master_df['t2_wins_alltime'][mask] = win_loss_list[3]
            master_df['t2_losses_alltime'][mask] = win_loss_list[4]
            master_df['t2_matches_alltime'][mask] = win_loss_list[5]  
            master_df['t2_win_percent_alltime'][mask] = t2_win_percent
            
    
                
    return(master_df)

In [27]:
def find_last_n_results(master_df, n):
    id_list = master_df.id.unique()
    for i in id_list:
        t1 = ((master_df[master_df['id'] == i]).team_one).iloc[0]
        t2 = ((master_df[master_df['id'] == i]).team_two).iloc[0]
        date = ((master_df[master_df['id'] == i]).date).iloc[0]

        teams=[t1, t2]
    
        win_loss_list=[]

        for team in teams:

            df_partial_season=master_df[master_df['date']<date]
            season_1 = df_partial_season[df_partial_season['team_one'] == team]
            season_2 = df_partial_season[df_partial_season['team_two'] == team]
                        
            #We need to combine season 1 and season 2 and sort by date
            team_df = pd.concat([season_1, season_2])
            #print(f"season_1: {season_1.shape} season_2: {season_2.shape} team_df: {team_df.shape}")
            team_df.sort_values(by=['date'], inplace=True, ascending=False)
            #display(team_df.head)
            top_n_team_df = team_df.head(n)
            
            wins = len(top_n_team_df[top_n_team_df['winner'] == team])
            losses = len(top_n_team_df[top_n_team_df['winner'] != team])
            matches = wins+losses
            #print(f"{team} {len(season_1)} {len(season_2)} {len(team_season)} {wins} ")
            if matches >= n:
                win_loss_list.append(wins)
                win_loss_list.append(losses)
                win_loss_list.append(matches)
            else:
                win_loss_list.append("")
                win_loss_list.append("")
                win_loss_list.append("")
            
            
        
        #print(return_info)
    
        mask = master_df['id'] == i

        #calculate the win percentages for each match
        if win_loss_list[2] == 0:
            t1_win_percent = 0
        elif win_loss_list[2] == "":
            t1_win_percent = ""
        else:
            t1_win_percent = win_loss_list[0] / win_loss_list[2]
        if win_loss_list[5] == 0:
            t2_win_percent = 0
        elif win_loss_list[5] == "":
            t2_win_percent = ""
        else:
            t2_win_percent = win_loss_list[3] / win_loss_list[5]
        
        if n == 3:
            master_df['t1_wins_last_3'][mask] = win_loss_list[0]
            master_df['t1_losses_last_3'][mask] = win_loss_list[1]
            master_df['t1_win_percent_last_3'][mask] = t1_win_percent
            master_df['t2_wins_last_3'][mask] = win_loss_list[3]
            master_df['t2_losses_last_3'][mask] = win_loss_list[4]
            master_df['t2_win_percent_last_3'][mask] = t2_win_percent
        if n == 5:
            master_df['t1_wins_last_5'][mask] = win_loss_list[0]
            master_df['t1_losses_last_5'][mask] = win_loss_list[1]
            master_df['t1_win_percent_last_5'][mask] = t1_win_percent
            master_df['t2_wins_last_5'][mask] = win_loss_list[3]
            master_df['t2_losses_last_5'][mask] = win_loss_list[4]
            master_df['t2_win_percent_last_5'][mask] = t2_win_percent
        elif n == 10:
            master_df['t1_wins_last_10'][mask] = win_loss_list[0]
            master_df['t1_losses_last_10'][mask] = win_loss_list[1]
            master_df['t1_win_percent_last_10'][mask] = t1_win_percent
            master_df['t2_wins_last_10'][mask] = win_loss_list[3]
            master_df['t2_losses_last_10'][mask] = win_loss_list[4]
            master_df['t2_win_percent_last_10'][mask] = t2_win_percent
        elif n == 20:
            master_df['t1_wins_last_20'][mask] = win_loss_list[0]
            master_df['t1_losses_last_20'][mask] = win_loss_list[1]
            master_df['t1_win_percent_last_20'][mask] = t1_win_percent
            master_df['t2_wins_last_20'][mask] = win_loss_list[3]
            master_df['t2_losses_last_20'][mask] = win_loss_list[4]
            master_df['t2_win_percent_last_20'][mask] = t2_win_percent    
    
    return master_df

In [28]:
results_2020 = find_team_match_results(df, '01/01/2020', '12/31/2020', 'current_season')
results_alltime = find_team_match_results(results_2020, '11/25/2010', '01/01/2080', 'all_time')
results_2019 = find_team_match_results(results_alltime, '01/01/2019', '12/31/2019', 'current_season')
results_2018 = find_team_match_results(results_2019, '01/01/2018', '12/31/2018', 'current_season')

In [29]:
results_last_5 = find_last_n_results(results_2018, 5)
results_last_10 = find_last_n_results(results_last_5, 10)
results_last_3 = find_last_n_results(results_last_10, 3)
results_last_20 = find_last_n_results(results_last_3, 20)
return_df = find_last_season_results(results_last_20)

In [30]:
return_df = find_head_to_head_results(return_df)

Team 1: Los Angeles Valiant Team 2: San Francisco Shock Date: 2018-01-11 00:00:00
Los Angeles Valiant vs San Francisco Shock on 2018-01-11 00:00:00 record: 0 - 0 
Team 1: Los Angeles Gladiators Team 2: Shanghai Dragons Date: 2018-01-11 00:00:00
Los Angeles Gladiators vs Shanghai Dragons on 2018-01-11 00:00:00 record: 0 - 0 
Team 1: Seoul Dynasty Team 2: Dallas Fuel Date: 2018-01-11 00:00:00
Seoul Dynasty vs Dallas Fuel on 2018-01-11 00:00:00 record: 0 - 0 
Team 1: Florida Mayhem Team 2: London Spitfire Date: 2018-01-11 00:00:00
Florida Mayhem vs London Spitfire on 2018-01-11 00:00:00 record: 0 - 0 
Team 1: Houston Outlaws Team 2: Philadelphia Fusion Date: 2018-01-11 00:00:00
Houston Outlaws vs Philadelphia Fusion on 2018-01-11 00:00:00 record: 0 - 0 
Team 1: New York Excelsior Team 2: Boston Uprising Date: 2018-01-12 00:00:00
New York Excelsior vs Boston Uprising on 2018-01-12 00:00:00 record: 0 - 0 
Team 1: Dallas Fuel Team 2: Los Angeles Valiant Date: 2018-01-13 00:00:00
Dallas Fuel 

Boston Uprising vs Houston Outlaws on 2018-02-10 00:00:00 record: 0 - 0 
Team 1: Philadelphia Fusion Team 2: Florida Mayhem Date: 2018-02-11 00:00:00
Philadelphia Fusion vs Florida Mayhem on 2018-02-11 00:00:00 record: 0 - 0 
Team 1: Houston Outlaws Team 2: London Spitfire Date: 2018-02-11 00:00:00
Houston Outlaws vs London Spitfire on 2018-02-11 00:00:00 record: 1 - 0 
Team 1: New York Excelsior Team 2: London Spitfire Date: 2018-02-11 00:00:00
New York Excelsior vs London Spitfire on 2018-02-11 00:00:00 record: 1 - 0 
Team 1: Los Angeles Valiant Team 2: Seoul Dynasty Date: 2018-02-22 00:00:00
Los Angeles Valiant vs Seoul Dynasty on 2018-02-22 00:00:00 record: 1 - 0 
Team 1: Shanghai Dragons Team 2: Dallas Fuel Date: 2018-02-22 00:00:00
Shanghai Dragons vs Dallas Fuel on 2018-02-22 00:00:00 record: 0 - 1 
Team 1: San Francisco Shock Team 2: Los Angeles Gladiators Date: 2018-02-22 00:00:00
San Francisco Shock vs Los Angeles Gladiators on 2018-02-22 00:00:00 record: 0 - 1 
Team 1: Londo

Seoul Dynasty vs Houston Outlaws on 2018-03-22 00:00:00 record: 1 - 0 
Team 1: Dallas Fuel Team 2: New York Excelsior Date: 2018-03-23 00:00:00
Dallas Fuel vs New York Excelsior on 2018-03-23 00:00:00 record: 0 - 1 
Team 1: Boston Uprising Team 2: San Francisco Shock Date: 2018-03-23 00:00:00
Boston Uprising vs San Francisco Shock on 2018-03-23 00:00:00 record: 0 - 1 
Team 1: Florida Mayhem Team 2: Seoul Dynasty Date: 2018-03-23 00:00:00
Florida Mayhem vs Seoul Dynasty on 2018-03-23 00:00:00 record: 0 - 1 
Team 1: Los Angeles Valiant Team 2: Philadelphia Fusion Date: 2018-03-24 00:00:00
Los Angeles Valiant vs Philadelphia Fusion on 2018-03-24 00:00:00 record: 1 - 0 
Team 1: Shanghai Dragons Team 2: London Spitfire Date: 2018-03-24 00:00:00
Shanghai Dragons vs London Spitfire on 2018-03-24 00:00:00 record: 0 - 1 
Team 1: London Spitfire Team 2: Dallas Fuel Date: 2018-03-24 00:00:00
London Spitfire vs Dallas Fuel on 2018-03-24 00:00:00 record: 1 - 0 
Team 1: New York Excelsior Team 2: Lo

Los Angeles Gladiators vs Florida Mayhem on 2018-05-03 00:00:00 record: 2 - 0 
Team 1: San Francisco Shock Team 2: Houston Outlaws Date: 2018-05-03 00:00:00
San Francisco Shock vs Houston Outlaws on 2018-05-03 00:00:00 record: 1 - 1 
Team 1: Florida Mayhem Team 2: San Francisco Shock Date: 2018-05-03 00:00:00
Florida Mayhem vs San Francisco Shock on 2018-05-03 00:00:00 record: 1 - 1 
Team 1: Los Angeles Valiant Team 2: Boston Uprising Date: 2018-05-04 00:00:00
Los Angeles Valiant vs Boston Uprising on 2018-05-04 00:00:00 record: 0 - 2 
Team 1: New York Excelsior Team 2: Dallas Fuel Date: 2018-05-04 00:00:00
New York Excelsior vs Dallas Fuel on 2018-05-04 00:00:00 record: 2 - 0 
Team 1: Boston Uprising Team 2: Los Angeles Gladiators Date: 2018-05-04 00:00:00
Boston Uprising vs Los Angeles Gladiators on 2018-05-04 00:00:00 record: 2 - 0 
Team 1: Shanghai Dragons Team 2: New York Excelsior Date: 2018-05-05 00:00:00
Shanghai Dragons vs New York Excelsior on 2018-05-05 00:00:00 record: 0 - 

Team 1: Philadelphia Fusion Team 2: San Francisco Shock Date: 2018-06-08 00:00:00
Philadelphia Fusion vs San Francisco Shock on 2018-06-08 00:00:00 record: 1 - 1 
Team 1: Shanghai Dragons Team 2: Boston Uprising Date: 2018-06-08 00:00:00
Shanghai Dragons vs Boston Uprising on 2018-06-08 00:00:00 record: 0 - 2 
Team 1: Dallas Fuel Team 2: Florida Mayhem Date: 2018-06-09 00:00:00
Dallas Fuel vs Florida Mayhem on 2018-06-09 00:00:00 record: 0 - 2 
Team 1: Seoul Dynasty Team 2: Philadelphia Fusion Date: 2018-06-09 00:00:00
Seoul Dynasty vs Philadelphia Fusion on 2018-06-09 00:00:00 record: 2 - 0 
Team 1: Los Angeles Gladiators Team 2: London Spitfire Date: 2018-06-09 00:00:00
Los Angeles Gladiators vs London Spitfire on 2018-06-09 00:00:00 record: 2 - 0 
Team 1: San Francisco Shock Team 2: New York Excelsior Date: 2018-06-09 00:00:00
San Francisco Shock vs New York Excelsior on 2018-06-09 00:00:00 record: 0 - 2 
Team 1: Los Angeles Valiant Team 2: Houston Outlaws Date: 2018-06-10 00:00:00


...

Toronto Defiant vs Vancouver Titans on 2019-05-04 00:00:00 record: 0 - 0 
Team 1: Shanghai Dragons Team 2: Washington Justice Date: 2019-05-04 00:00:00
Shanghai Dragons vs Washington Justice on 2019-05-04 00:00:00 record: 0 - 0 
Team 1: London Spitfire Team 2: Houston Outlaws Date: 2019-05-04 00:00:00
London Spitfire vs Houston Outlaws on 2019-05-04 00:00:00 record: 1 - 4 
Team 1: Atlanta Reign Team 2: New York Excelsior Date: 2019-05-04 00:00:00
Atlanta Reign vs New York Excelsior on 2019-05-04 00:00:00 record: 1 - 0 
Team 1: Seoul Dynasty Team 2: Vancouver Titans Date: 2019-05-04 00:00:00
Seoul Dynasty vs Vancouver Titans on 2019-05-04 00:00:00 record: 0 - 2 
Team 1: Chengdu Hunters Team 2: Los Angeles Valiant Date: 2019-05-05 00:00:00
Chengdu Hunters vs Los Angeles Valiant on 2019-05-05 00:00:00 record: 0 - 0 
Team 1: Toronto Defiant Team 2: Paris Eternal Date: 2019-05-05 00:00:00
Toronto Defiant vs Paris Eternal on 2019-05-05 00:00:00 record: 0 - 0 
Team 1: Washington Justice Team 

London Spitfire vs Washington Justice on 2019-06-22 00:00:00 record: 1 - 0 
Team 1: Atlanta Reign Team 2: Philadelphia Fusion Date: 2019-06-22 00:00:00
Atlanta Reign vs Philadelphia Fusion on 2019-06-22 00:00:00 record: 0 - 2 
Team 1: Toronto Defiant Team 2: Guangzhou Charge Date: 2019-06-22 00:00:00
Toronto Defiant vs Guangzhou Charge on 2019-06-22 00:00:00 record: 0 - 0 
Team 1: Hangzhou Spark Team 2: Seoul Dynasty Date: 2019-06-23 00:00:00
Hangzhou Spark vs Seoul Dynasty on 2019-06-23 00:00:00 record: 1 - 0 
Team 1: Chengdu Hunters Team 2: Dallas Fuel Date: 2019-06-23 00:00:00
Chengdu Hunters vs Dallas Fuel on 2019-06-23 00:00:00 record: 1 - 0 
Team 1: Boston Uprising Team 2: Paris Eternal Date: 2019-06-23 00:00:00
Boston Uprising vs Paris Eternal on 2019-06-23 00:00:00 record: 0 - 1 
Team 1: Houston Outlaws Team 2: Florida Mayhem Date: 2019-06-23 00:00:00
Houston Outlaws vs Florida Mayhem on 2019-06-23 00:00:00 record: 5 - 0 
Team 1: Los Angeles Valiant Team 2: Vancouver Titans Dat

Shanghai Dragons vs Toronto Defiant on 2019-08-02 00:00:00 record: 0 - 0 
Team 1: Los Angeles Valiant Team 2: Seoul Dynasty Date: 2019-08-02 00:00:00
Los Angeles Valiant vs Seoul Dynasty on 2019-08-02 00:00:00 record: 3 - 2 
Team 1: New York Excelsior Team 2: Paris Eternal Date: 2019-08-02 00:00:00
New York Excelsior vs Paris Eternal on 2019-08-02 00:00:00 record: 1 - 0 
Team 1: Houston Outlaws Team 2: Philadelphia Fusion Date: 2019-08-03 00:00:00
Houston Outlaws vs Philadelphia Fusion on 2019-08-03 00:00:00 record: 1 - 4 
Team 1: Chengdu Hunters Team 2: Los Angeles Gladiators Date: 2019-08-03 00:00:00
Chengdu Hunters vs Los Angeles Gladiators on 2019-08-03 00:00:00 record: 0 - 1 
Team 1: Guangzhou Charge Team 2: Hangzhou Spark Date: 2019-08-03 00:00:00
Guangzhou Charge vs Hangzhou Spark on 2019-08-03 00:00:00 record: 0 - 1 
Team 1: Atlanta Reign Team 2: Paris Eternal Date: 2019-08-03 00:00:00
Atlanta Reign vs Paris Eternal on 2019-08-03 00:00:00 record: 1 - 0 
Team 1: Florida Mayhem T

Seoul Dynasty vs Hangzhou Spark on 2019-09-08 00:00:00 record: 0 - 2 
Team 1: London Spitfire Team 2: San Francisco Shock Date: 2019-09-08 00:00:00
London Spitfire vs San Francisco Shock on 2019-09-08 00:00:00 record: 4 - 1 
Team 1: Los Angeles Gladiators Team 2: Vancouver Titans Date: 2019-09-08 00:00:00
Los Angeles Gladiators vs Vancouver Titans on 2019-09-08 00:00:00 record: 0 - 2 
Team 1: Atlanta Reign Team 2: New York Excelsior Date: 2019-09-08 00:00:00
Atlanta Reign vs New York Excelsior on 2019-09-08 00:00:00 record: 2 - 0 
Team 1: Atlanta Reign Team 2: Hangzhou Spark Date: 2019-09-12 00:00:00
Atlanta Reign vs Hangzhou Spark on 2019-09-12 00:00:00 record: 1 - 0 
Team 1: Los Angeles Gladiators Team 2: San Francisco Shock Date: 2019-09-13 00:00:00
Los Angeles Gladiators vs San Francisco Shock on 2019-09-13 00:00:00 record: 4 - 2 
Team 1: New York Excelsior Team 2: Vancouver Titans Date: 2019-09-13 00:00:00
New York Excelsior vs Vancouver Titans on 2019-09-13 00:00:00 record: 0 - 2

Washington Justice vs Dallas Fuel on 2020-04-11 00:00:00 record: 0 - 1 
Team 1: Chengdu Hunters Team 2: Vancouver Titans Date: 2020-04-12 00:00:00
Chengdu Hunters vs Vancouver Titans on 2020-04-12 00:00:00 record: 0 - 2 
Team 1: Shanghai Dragons Team 2: Guangzhou Charge Date: 2020-04-12 00:00:00
Shanghai Dragons vs Guangzhou Charge on 2020-04-12 00:00:00 record: 2 - 1 
Team 1: Houston Outlaws Team 2: Toronto Defiant Date: 2020-04-12 00:00:00
Houston Outlaws vs Toronto Defiant on 2020-04-12 00:00:00 record: 2 - 1 
Team 1: Philadelphia Fusion Team 2: Atlanta Reign Date: 2020-04-12 00:00:00
Philadelphia Fusion vs Atlanta Reign on 2020-04-12 00:00:00 record: 3 - 0 
Team 1: San Francisco Shock Team 2: Los Angeles Gladiators Date: 2020-04-13 00:00:00
San Francisco Shock vs Los Angeles Gladiators on 2020-04-13 00:00:00 record: 3 - 5 
Team 1: Atlanta Reign Team 2: Washington Justice Date: 2020-04-16 00:00:00
Atlanta Reign vs Washington Justice on 2020-04-16 00:00:00 record: 2 - 0 
Team 1: Los 

In [31]:
return_df.to_csv('week-12-complete.csv')